In [1]:
import ROOT
import pandas as pd
import numpy as np
import os
import sys
import ipynbname
from pathlib import Path

project_root = str(ipynbname.path().parent.parent)
sys.path.append(project_root)

from processing import SiphraAcquisition, fit_peak_expbg

# ROOT.gROOT.SetBatch(False)
# ROOT.gEnv.SetValue("Canvas.ShowEventStatus", 1)
# ROOT.gEnv.SetValue("Canvas.ShowToolBar", 1)
# ROOT.gEnv.SetValue("Canvas.UseGL", 0)
# ROOT.gROOT.ProcessLine("gVirtualX = new TGX11();")


In [2]:
# Constants
BITS12 = 2**12
BITS9 = 2**9 # 512 typical number of bins used
# Energy emission peaks in MeV
K40_MEV = 1.460
TL208_MEV = 2.614

In [13]:
BG_PMchs58 = []
BG_PMchs58.append(SiphraAcquisition(project_root+'/data/260209/1_SiPM_ChannelsTest_Ch5-8_Ch2_CT_Thr1_cmisvoffset127_Background.csv',
                                      active_chs=2,
                                      exposure_sec=119.634,
                                      sipm_chs="5-8"))
BG_PMchs58.append(SiphraAcquisition(project_root+'/data/260209/2_SiPM_ChannelsTest_Ch5-8_Ch6_CT_Thr1_cmisvoffset127_Background.csv',
                                      active_chs=6,
                                      exposure_sec=161.062,
                                      sipm_chs="5-8"))
BG_PMchs58.append(SiphraAcquisition(project_root+'/data/260209/3_SiPM_ChannelsTest_Ch5-8_Ch10_CT_Thr1_cmisvoffset127_Background.csv',
                                      active_chs=10,
                                      exposure_sec=148.920,
                                      sipm_chs="5-8"))
BG_PMchs58.append(SiphraAcquisition(project_root+'/data/260209/4_SiPM_ChannelsTest_Ch5-8_Ch14_CT_Thr1_cmisvoffset127_Background.csv',
                                      active_chs=14,
                                      exposure_sec=164.828,
                                      sipm_chs="5-8"))

BG_PMchs14 = []
BG_PMchs14.append(SiphraAcquisition(project_root+'/data/260209/5_SiPM_ChannelsTest_Ch1-4_Ch2_CT_Trh1_cmisvoffset127_Background.csv',
                                      active_chs=2,
                                      exposure_sec=34.836,
                                      sipm_chs="1-4"))
BG_PMchs14.append(SiphraAcquisition(project_root+'/data/260209/6_SiPM_ChannelsTest_Ch1-4_Ch6_CT_Thr1_cmisvoffset127_Background.csv',
                                      active_chs=6,
                                      exposure_sec=80.792,
                                      sipm_chs="1-4"))
BG_PMchs14.append(SiphraAcquisition(project_root+'/data/260209/7_SiPM_ChannelsTest_Ch1-4_Ch10_CT_Thr1_cmisvoffset127_Background.csv',
                                      active_chs=10,
                                      exposure_sec=54.386,
                                      sipm_chs="1-4"))
BG_PMchs14.append(SiphraAcquisition(project_root+'/data/260209/8_SiPM_ChannelsTest_Ch1-4_Ch14_CT_Thr1_cmisvoffset127_Background.csv',
                                      active_chs=14,
                                      exposure_sec=90.837,
                                      sipm_chs="1-4"))

Cs137_PMchs14 = []
Cs137_PMchs14.append(SiphraAcquisition(project_root+'/data/260209/12_SiPM_ChannelsTest_Ch1-4_Ch2_CT_Thr1_cmisvoffset127_Cs137.csv',
                                      active_chs=2,
                                      exposure_sec=23.381,
                                      sipm_chs="1-4"))
Cs137_PMchs14.append(SiphraAcquisition(project_root+'/data/260209/11_SiPM_ChannelsTest_Ch1-4_Ch6_CT_Thr1_cmisvoffset127_Cs137.csv',
                                      active_chs=6,
                                      exposure_sec=24.606,
                                      sipm_chs="1-4"))
Cs137_PMchs14.append(SiphraAcquisition(project_root+'/data/260209/10_SiPM_ChannelsTest_Ch1-4_Ch10_CT_Thr1_cmisvoffset127_Cs137.csv',
                                      active_chs=10,
                                      exposure_sec=24.144,
                                      sipm_chs="1-4"))
Cs137_PMchs14.append(SiphraAcquisition(project_root+'/data/260209/9_SiPM_ChannelsTest_Ch1-4_Ch14_CT_Thr1_cmisvoffset127_Cs137.csv',
                                      active_chs=14,
                                      exposure_sec=24.645,
                                      sipm_chs="1-4"))

Cs137_PMchs58 = []
Cs137_PMchs58.append(SiphraAcquisition(project_root+'/data/260209/13_SiPM_ChannelsTest_Ch5-8_Ch2_CT_Thr1_cmisvoffset127_Cs137.csv',
                                      active_chs=2,
                                      exposure_sec=24.720,
                                      sipm_chs="5-8"))
Cs137_PMchs58.append(SiphraAcquisition(project_root+'/data/260209/14_SiPM_ChannelsTest_Ch5-8_Ch6_CT_Thr1_cmisvoffset127_Cs137.csv',
                                      active_chs=6,
                                      exposure_sec=24.916,
                                      sipm_chs="5-8"))
Cs137_PMchs58.append(SiphraAcquisition(project_root+'/data/260209/15_SiPM_ChannelsTest_Ch5-8_Ch10_CT_Thr1_cmisvoffset127_Cs137.csv',
                                      active_chs=10,
                                      exposure_sec=24.848,
                                      sipm_chs="5-8"))
Cs137_PMchs58.append(SiphraAcquisition(project_root+'/data/260209/16_SiPM_ChannelsTest_Ch5-8_Ch14_CT_Thr1_cmisvoffset127_Cs137.csv',
                                      active_chs=14,
                                      exposure_sec=24.934,
                                      sipm_chs="5-8"))

datasets_lsts = [BG_PMchs58, Cs137_PMchs58, BG_PMchs14, Cs137_PMchs14]
colors = [ROOT.kRed, ROOT.kSpring, ROOT.kBlue, ROOT.kOrange, ROOT.kCyan, ROOT.kYellow, ROOT.kGreen, ROOT.kViolet]

In [4]:
current_ds = datasets_lsts[2][0]
print(current_ds.filepath.name)
current_ds[16]

5_SiPM_ChannelsTest_Ch1-4_Ch2_CT_Trh1_cmisvoffset127_Background.csv


array([137., 135., 135., ..., 135., 136., 135.], shape=(99976,))

In [14]:
canvas_raw = []
cv_names = ["canvas_raw_"+str(_) for _ in range(1, 5)]
for name in cv_names:
    if ROOT.gROOT.FindObject(name):
        ROOT.gROOT.FindObject(name).Close()
ROOT.gStyle.SetOptStat(11)
ROOT.gStyle.SetStatFontSize(0.03)
ROOT.gStyle.SetStatW(0.15)

legends_raw = [ROOT.TLegend(0.65, 0.7, 0.8, 0.88) for _ in range(1,5)]
Yinit = 0.82 # For stat boxes

hists_raw_dict = {}
hists_raw_names = ["hist58Bg", "hist58Cs", "hist14Bg", "hist14Cs"]

Yinit = 0.82 # For stat boxes

for dataset, name, cv, lg in zip(datasets_lsts, hists_raw_names, cv_names, legends_raw):
    hists_raw_dict[name] = []
    current_hist_list = hists_raw_dict[name]
    canvas_raw.append(ROOT.TCanvas(cv, cv, 800, 600))
    lg.SetHeader('SIPHRA channel')
    # legends_raw.append(ROOT.TLegend(0.65, 0.7, 0.8, 0.88))
    for idx, (acq, color) in enumerate(zip(dataset, colors)):
        # print(f"Current file: {acq.filepath.name}")
        ch = acq.active_chs[0]
        hist = ROOT.TH1F(f"{name}Ch{ch}", "", BITS9, 0, BITS12)
        hist.Fill(acq[ch])
        hist.Scale(1/acq.exposure)
        #Preeting up..
        hist.GetXaxis().SetTitle("ADC channel number")
        hist.GetYaxis().SetTitle(r"Count rate (s^{-1})")
        hist.SetLineColor(color+2)
        hist.SetTitle(name)
        lg.AddEntry(hist, f"Ch. {ch}", 'l')
        current_hist_list.append(hist)
        current_hist_list[-1].Draw('sames hist')
    canvas_raw[-1].SetLogy()
    ROOT.gPad.Update()
    for i, sp in enumerate(current_hist_list):
        st = sp.FindObject("stats")
        # print(type(st))
        st.SetY1NDC(Yinit - i*0.08)
        st.SetY2NDC(0.1)
    legends_raw[-1].Draw()
    canvas_raw[-1].Draw()

Warning in <TROOT::Append>: Replacing existing TH1: hist58BgCh2 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hist58BgCh6 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hist58BgCh10 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hist58BgCh14 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hist58CsCh2 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hist58CsCh6 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hist58CsCh10 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hist58CsCh14 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hist14BgCh2 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hist14BgCh6 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hist14BgCh10 (Potential memory leak).
Warning in <TROO

In [8]:
# if ROOT.gROOT.FindObject("canvas"):
#     ROOT.gROOT.FindObject("canvas").Close()
# canvas = ROOT.TCanvas("canvas", "canvas", 800, 600)
# ROOT.gStyle.SetOptStat(11)
# ROOT.gStyle.SetStatFontSize(0.03)
# ROOT.gStyle.SetStatW(0.15)
# legend_ary = ROOT.TLegend(0.65, 0.7, 0.8, 0.88)

# Yinit = 0.82 # For stat boxes
# hists_58BG = []
# current_dataset = datasets_lsts[2]
# for acq, color in zip(current_dataset, colors):
#     hist = ROOT.TH1F(f"h58bgCh{acq.active_chs[0]}", "", BITS9, 0, BITS12)
#     hist.Fill(acq[acq.active_chs[0]])
#     hist.Scale(1/acq.exposure)
#     # Preeting up...
#     hist.GetXaxis().SetTitle("ADC channel number")
#     hist.GetYaxis().SetTitle(r"Count rate (s^{-1})")
#     hist.SetLineColor(color+2)
#     legend_ary.SetHeader('SIPHRA channel')
#     legend_ary.AddEntry(hist, f"Ch. {acq.active_chs[0]}", 'l')
#     hists_58BG.append(hist)
#     hists_58BG[-1].Draw("sames hist")
# canvas.SetLogy()

# ROOT.gPad.Update()
# Yinit = 0.82
# for i, sp in enumerate(hists_58BG):
#     st = sp.FindObject("stats")
#     print(type(st))
#     st.SetY1NDC(Yinit - i*0.08)
#     st.SetY2NDC(0.1)


# legend_ary.Draw()
# canvas.Draw()
# ROOT.gPad.Modified()
# ROOT.gPad.Update()

In [7]:
canvas = ROOT.TCanvas("canvas", "canvas", 800, 600)
for hist in hists_58BG:
    fit_fn, fit_result = fit_peak_expbg(hist=hist, name='k40peak', xl=700, xr=1300, norm=1, mean=870, showFit=True, keep_prev_fncs=False)
    # hist.Draw('sames hist')
for hist in hists_58BG: # If this is inside the first for loop, nothing is plotted
    hist.Draw('sames hist')
canvas.SetLogy()
canvas.Draw()
print(fit_fn.GetParameter('Mean'))

864.490254507108
****************************************
Minimizer is Minuit2 / Migrad
MinFCN                    =     0.454998
Chi2                      =     0.909996
NDf                       =           71
Edm                       =  4.19258e-06
NCalls                    =          254
Const                     =      55.1644   +/-   67.0627     
Denom                     =      212.972   +/-   54.3249     
Norm                      =     0.990514   +/-   0.486902    
Mean                      =      863.094   +/-   44.5963     
Sigma                     =     -68.8679   +/-   41.554      
****************************************
Minimizer is Minuit2 / Migrad
MinFCN                    =     0.361677
Chi2                      =     0.723354
NDf                       =           71
Edm                       =  9.93854e-07
NCalls                    =          237
Const                     =      58.8242   +/-   68.9613     
Denom                     =      214.571   +/-   52.9045   

Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas


In [ ]:
for element in hists_raw_dict.values():
    print(element)

In [ ]:
from collections import namedtuple

BGPeaks = namedtuple('BGPeaks', ('k40', 'cs137'))

for hist_list in [hists_raw_dict[hists_58BG], hists_raw_dict[hists_]:
    for hist in hist_list:
        fit_peak_expbg(hist=hist, name='k40peak', xl=700, xr=1300, norm=1, mean=870, showFit=False, keep_prev_fncs=True)
        fit_peak_expbg(hist=hist, name='cs137peak', xl=1200, xr=1800, norm=0.3, mean=1400, showFit=False, keep_prev_fncs=True)
        